In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import glob 
from matplotlib import cm

def set_fig():     
    
    plt.rc('xtick', labelsize=40) 
    plt.rc('ytick', labelsize=40) 
    plt.rc('axes', labelsize=60) # fontsize of the x and y labels
    
#    ax.xaxis.set_minor_locator(AutoMinorLocator(2))  
#    ax.yaxis.set_minor_locator(AutoMinorLocator(2))

    ax.tick_params(which='major', bottom=True, top=True, left=True, right=True, length=15, width=3, direction="in", pad=20)
    ax.tick_params(which='minor', bottom=True, top=True, left=True, right=True, length=10, width=3, direction="in", pad=20)
    
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(3)
        
plt.rcParams['figure.figsize'] = [10, 8]

In [ ]:
for file in glob.glob("*cell"):
    cell_file = file

for file in glob.glob("*ener"):
    ener_file = file

for file in glob.glob("*vdw*"):
    vdw_file = file

print(ener_file, cell_file)

cell = np.genfromtxt(cell_file, skip_header=1 )
cell = cell.astype(float)
ener = np.genfromtxt(ener_file, skip_header=1 )
ener= ener.astype(float)

In [ ]:
cx = 1
cy = 11

ex = 1
ey = 4

fig, axs = plt.subplots(2)

c_heading =["Steps", "Time (fs)", "a (angstrom)", "ay", "az", "bx", " b (angstrom)", "bz", "cx", "cy", "c (angstrom)", "Volume (cubic angstrom)"]
e_heading =["Steps", "Time (fs)", "Kin. E. (a.u.)", "Temperature (K)", "Pot. E (a.u.)", "Cons. Qty. (a.u.)", "Time per Step (s)"]

axs[0].set(ylabel=c_heading[cy], xlabel=c_heading[cx])
axs[1].set(ylabel=e_heading[ey], xlabel=c_heading[ex])

axs[0].plot(cell[:,cx], cell[:,cy], color="red", label=c_heading[cy])
axs[1].plot(ener[:,ex], ener[:,ey], color="blue", label=e_heading[ey])


In [ ]:
plt.xlabel('Volume (Angstrom^3)', labelpad=20,  fontsize = 15)
plt.ylabel('Potential Energy (kJ/mol)', labelpad=20, fontsize = 15)

plt.show

In [ ]:
nstep = cell[-1,0]
nstep = int(nstep)

nbins = 25

mi = np.min(cell[:,-1])
ma = np.max(cell[:,-1])
change = ma - mi 
bin_size = change/nbins

vols = np.arange(mi, ma, bin_size)

steps = np.zeros((2,nbins))

for ix in range(nbins):
    for iy in range(nstep):
        if int(cell[iy,-1]) == int(vols[ix]):

            steps[0, ix] = cell[iy,0]
            steps[1,ix] = cell[iy,-1]
            break
        elif int(cell[iy,-1]) - 5 < int(vols[ix]) < int(cell[iy,-1])+5:
            steps[0, ix] = cell[iy,0]
            steps[1,ix] = cell[iy,-1]
            break

eners = np.zeros((2,nbins))

for ix in range(nbins):
    eners[0,ix] = ener[int(steps[0,ix]),4]
    eners[1,ix] = ener[int(steps[0,ix]),5]
        
# Error bars 

st = np.zeros((nbins))

for ix in range(nbins):
    en = []
    vo = []

    top = vols[ix] + (bin_size/2)
    bot = vols[ix] - (bin_size/2)

    for iy in range(nstep):

        if bot <= (cell[iy,-1]) <= top:
            vo.append(float(cell[iy,-1]))
            en.append(float(ener[iy,4]))

    st[ix] = (np.std(en))*2625


# Plotting with error bars 

plt.xlabel('Volume (Angstrom^3)', labelpad=20,  fontsize = 15)
plt.ylabel('Potential Energy (kJ/mol)', labelpad=20, fontsize = 15)

plt.scatter((steps[1,:]), (eners[0,:]*2625)-min(eners[0,:]*2625), color="red", s = 50)

plt.errorbar((steps[1,:]),(eners[0,:]*2625)-min(eners[0,:]*2625),yerr=st[:],color="black", fmt="o")
plt.plot((steps[1,:]), (eners[0,:]*2625)-min(eners[0,:]*2625), color="black")

In [ ]:
a = np.zeros((nbins,3))

for ix in range(nbins):
    a[ix,0] = steps[1,ix]
    a[ix,1] = eners[0,ix]*2625
    a[ix,2] = st[ix]

np.savetxt("ambient_PMF.txt", a)